In [12]:
using LinearAlgebra

# Extra stuff

In [13]:
mutable struct curv
    c # Curve
    a # Start value  
    b # End value
    w # Weights for integration
    N # Amount of
    M # amount of elements
    q # element spacing
end  

In [14]:
function grid_weight_find(a,b)
    
    J = Matrix(SymTridiagonal(a,b[1:(end-1)]))
    
    xgrid, U = eigen(J);
    
    U = transpose(sign.(U[1,:])) .* U
    
    w = abs.(U[1,:]).^2
    
    return xgrid, w, U
    
end

grid_weight_find (generic function with 1 method)

# The quads

In [15]:
function stand_int(f,s)
    
    # function f 
    
    #object s with: curve - c
    #               start value - a
    #               end value - b
    #               curve derivative - w
    
    f_curv = x-> map(f,map(s.c,x))  .* map(s.w,x); # map to real line
    
    Dtrans_func = (s.b-s.a)/2;
    trans_func = x -> (s.b + s.a)/2 + x * (s.b - s.a)/2 ;
    
    g = x -> f_curv(trans_func(x)) * Dtrans_func; # map to [-1,1]
    return g
end

stand_int (generic function with 1 method)

In [16]:
function Clen_Curt(f,s)
    
    # function f
    
    #object s with: curve - c
    #               start value - a
    #               end value - b
    #               curve derivative - w
    #               number of quadrature points - N
    
    N = s.N

    if mod(N,2) != 0
       N -= 1 
    end
    
    f_int = stand_int(f,s)
    n = 0:N/2;
    D = 2 * cos.(2* transpose(n) .* n * pi/N)/N;
    D[1,:] = D[1,:] .* 0.5;
    d = [1; 2 ./ (1 .- (2:2:N).^2)];
    w = D * d;
    x = cos.( (0:N) * π / N );
    w = [w;w[length(w)-1:-1:1]];
    res = sum(map(f_int,x) .* w)
    
    return res
end

Clen_Curt (generic function with 1 method)

In [17]:
function m_Filon_Clen_Curt(f_o,s)
    
    # function f
    
    #object s with: curve - c
    #               start value - a
    #               end value - b
    #               curve derivative - w
    #               number of segments - M
    #               Spacing factor - q
    #               number of quadrature points - N (per segment)
    
    M = s.M
    N = s.N 
    q = s.q

    if mod(N,2) != 0
       N -= 1 
    end
    
    f = x-> map(f_o,map(s.c,x))  .* map(s.w,x); # map to real line

    mua = s.a * ((1:M) ./ M).^q;
    mub = s.b * ((1:M) ./ M).^q;
    
    resa = 0;
    resb = 0;

    for i1 = 1:(M-1)
        sa = curv(x->x,mua[i1],mua[i1+1],x->1)
        sb = curv(x->x,mub[i1],mub[i1+1],x->1)
        resa = resa + Clen_Curt(f,sa,N);
        resb = resb + Clen_Curt(f,sb,N);
    end

    if s.a == 0
        resa = 0;
    end
    if s.b == 0
        resb = 0;
    end
    
    res = resb-resa;
    
    return res
end

m_Filon_Clen_Curt (generic function with 1 method)

In [18]:
function Trap(f_0,s)
    
    #Good for periodic functions
    
    N = s.N 
    
    f = x-> map(f_0,map(s.c,x))  .* map(s.w,x); # map to real line
    
    dx = (s.b-s.a)/(N);
    
    res = (f(s.a)+f(s.b))/2 * dx
    
    for i1 = 1:N-1
        res = dx * f((s.b-s.a) * i1 / N + s.a)
    end
    
    return res
end

Trap (generic function with 1 method)

In [19]:
function Laguerre_quad(f_o,s)
    
    #
    # For integrals of the the ∫_0^∞ exp(-x)*g(x)dx, here f(x) = exp(x)*g(x)
    # Problem: we need the poly function and it can't go very high
    #
    
    # xL_k = (2k+1)L_k - kL_{k-1} - (k+1)L_{k+1}, k ≥ 1
    
    N = s.N 
    
    f = x -> map(f_o,map(s.c,x)) .* exp(x)  .* map(s.w,x); # map to real line
    
    K = 1:N
    
    a = collect(2 .* K .- 1) 
    b = collect(-K)
    x, w, uu = Cheb_x_w(a,b)
    
    res = sum(w .* map(f,x))
    
    return res
end

Laguerre_quad (generic function with 1 method)

In [20]:
function two_sided_Laguerre_quad(f_o,s)
    
    #
    # For integrals of the the ∫_0^∞ exp(-x)*g(x)dx, here f(x) = exp(-x)*g(x)
    # Problem: we need the poly function and it can't go very high
    #
    
    # xL_k = (2k+1)L_k - kL_{k-1} - (k+1)L_{k+1}, k ≥ 1
    
    N = s.N 

    f = x -> map(f_o,map(s.c,x)) .* exp(x)  .* map(s.w,x); # map to real line
    f_oppo = x -> map(f_o,map(s.c,-x)) .* exp(x)  .* map(s.w,-x); # map to real line
    
    K = 1:N
    
    a = collect(2 .* K .- 1) 
    b = collect(-K)
    x, w, uu = grid_weight_find(a,b)
    
    res = sum(w .* map(f,x)) + sum(w .* map(f_oppo,x))
    
    return res
end

two_sided_Laguerre_quad (generic function with 1 method)

In [21]:
function Hermite_quad(f_o,s)
    
    #
    # For integrals of the the ∫_{-∞}^∞ exp(-x^2)*g(x)dx, here f(x) = exp(-x^2)*g(x)
    # Problem: Only takes like N = 5, works good, but very variable
    #
    
    # x H_n = 1/2H_{n+1}+nH_{n-1}
    # 1/2H_{n+1} = x H_n-nH_{n-1}
    
    N = s.N 

    f = x -> map(f_o,map(s.c,x)) .* exp(x .^ 2)  .* map(s.w,x); # map to real line

    a = zeros(N);
    b = sqrt.((1:N) ./ 2)
    x, w, uu = grid_weight_find(a,b)
    
    w *= sqrt(pi) # multiply by int of e^(-x^2)

    res = sum(w .* map(f,x))
    
    return res

end

Hermite_quad (generic function with 1 method)

In [22]:
using NBInclude
nbexport("myquad.jl", "myquad.ipynb")